本项目将SVM作为一个回归器（在之前项目中是作为分类器使用）来估算交通流量     
在traffic_data.txt中包含的数据集统计了球队进行主场比赛期间，体育场旁边马路通过的车辆数量，如下：
- 星期
- 时间
- 对手球队
- 棒球比赛是否正在继续
- 通行的汽车数量

In [1]:
from sklearn.svm import SVC
from sklearn import preprocessing
import numpy as np

import warnings
warnings.filterwarnings("ignore")

def txt_read2(filename,delim):
    x_data = []
    with open(filename,'r') as f:
        for line in f:
            data = line.strip().split(delim)
            x_data.append(data)

    x_data = np.array(x_data)
    return x_data

def encode(x_data):
    # 将字符串转化为数值
    label_encoder = []
    x_encoded = np.empty(x_data.shape)
    for i,item in enumerate(x_data[0]):
        if item.isdigit():
            x_encoded[:,i] = x_data[:,i]
        else:
            label_encoder.append(preprocessing.LabelEncoder())
            x_encoded[:,i] = label_encoder[-1].fit_transform(x_data[:,i])
            
    x_values = x_encoded[:,:-1].astype(int)
    y_values = x_encoded[:,-1].astype(int)
    print(y_values)
    return x_values,y_values,label_encoder

def svm(params,x_data,y_data):
    from sklearn.model_selection import train_test_split
    import sklearn.metrics as sm

    x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.25,random_state=5)
    regressor = SVC(**params)
    regressor.fit(x_train,y_train)
    
    y_test_pred = regressor.predict(x_test)
    print('均方误差 = ',round(sm.mean_squared_error(y_test,y_test_pred),2))
    
    return regressor

def testone(regressor,input_data):
    count = 0
    input_data_encoded = [-1]*len(input_data)
    for i,item in enumerate(input_data):
        contain = []
        contain.append(item)
        if item.isdigit():
            input_data_encoded[i] = int(item) # 注意，如果不化为int型则会报错
        else:    
            input_data_encoded[i] = int(label_encoder[count].transform(contain)) 
            count += 1
        
    input_data_encoded = np.array(input_data_encoded).reshape(1,-1)

    print('Predicted traffic:',int(regressor.predict(input_data_encoded)[0]))

x_data = txt_read2('traffic_data.txt',',')
x_values,y_values,label_encoder = encode(x_data)
params = {'kernel':'rbf','C':10.0}
regressor = svm(params,x_values,y_values)
input_data = ['Tuesday','13:35','San Francisco','yes']
testone(regressor,input_data )


[ 3  8 10 ... 11 14 17]
均方误差 =  97.35
Predicted traffic: 32
